<a href="https://colab.research.google.com/github/JBlizzard-sketch/LoanIQ/blob/main/LoanIQFinalFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------------
# Heavy-version Cell 1
# (Bootstrap + write core modules)
# Paste & run in a FRESH Colab runtime.
# -------------------------

NGROK_TOKEN = "31rYvgklL0EdX9bGLvTXc313efE_2GyDFGPUNAyFgB83bikTF"  # kept for later cells

import sys, subprocess, importlib.util, os, time, shutil, textwrap, json, hashlib

# 1) Install packages if missing (prefer binary wheels; fallback to source if necessary)
def ensure_packages():
    pkgs = [
        "streamlit==1.37.0",
        "pyngrok==7.2.0",
        "pandas==2.2.2",
        "numpy==1.26.4",
        "scikit-learn==1.2.2",
        "xgboost",   # prefer latest compatible
        "shap",      # explainability
        "imbalanced-learn",
        "Faker==25.8.0",
        "joblib"
    ]
    # check a sentinel package (streamlit) first to skip reinstallation
    if importlib.util.find_spec("streamlit") is None:
        print("Installing packages (prefer binary wheels; may fallback to source)...")
        cmd = [sys.executable, "-m", "pip", "install", "--quiet", "--prefer-binary"] + pkgs
        subprocess.check_call(cmd)
    else:
        print("Streamlit already installed; skipping package install (fast path).")

ensure_packages()

# 2) Create project layout
ROOT = "/content/loan_app_full"
if os.path.exists(ROOT):
    print("Removing existing path for a clean bootstrap:", ROOT)
    shutil.rmtree(ROOT)
os.makedirs(ROOT, exist_ok=True)
os.makedirs(os.path.join(ROOT, "modules"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "modules", "ml"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "modules", "synth"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "data"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "models"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "logs"), exist_ok=True)

# small helper to write files and normalize curly quotes/unicode
def safe_write(path, content):
    s = content.replace("\r\n", "\n").replace("\r", "\n")
    for a,b in [("\u201c","\""),("\u201d","\""),("\u2018","'"),("\u2019","'"),("\u2013","-"),("\u2014","-")]:
        s = s.replace(a,b)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(s)
    print("Wrote:", path)

# -------------------------
# 3) modules/auth.py (kept simple as requested, but robust session helpers)
# -------------------------
auth_py = r'''
"""
auth.py - simple SQLite auth used for demo/testing.
Note: per user request this uses SHA-256 without salt for now (will be improved later).
"""
import os, sqlite3, hashlib
DB_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "data", "users.db"))

def _conn():
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    return sqlite3.connect(DB_PATH, check_same_thread=False)

def init_db():
    conn = _conn(); cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password_hash TEXT,
        role TEXT DEFAULT 'user'
    )""")
    conn.commit()
    # create default admin
    cur.execute("SELECT 1 FROM users WHERE username = ?", ("admin",))
    if cur.fetchone() is None:
        cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                    ("admin", sha256("admin123"), "admin"))
        conn.commit()
    conn.close()

def sha256(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

def register_user(username: str, password: str, role: str = "user"):
    conn = _conn(); cur = conn.cursor()
    try:
        cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                    (username, sha256(password), role))
        conn.commit()
        return True, "ok"
    except sqlite3.IntegrityError:
        return False, "username exists"
    except Exception as e:
        return False, str(e)
    finally:
        conn.close()

def check_login(username: str, password: str):
    conn = _conn(); cur = conn.cursor()
    cur.execute("SELECT password_hash, role FROM users WHERE username = ?", (username,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return False, None
    stored_hash, role = row
    if stored_hash == sha256(password):
        return True, role
    return False, None
'''
safe_write(os.path.join(ROOT, "modules", "auth.py"), auth_py)

# -------------------------
# 4) modules/schema.py (feature derivation and validation)
# -------------------------
schema_py = r'''
"""
schema.py - prepares a DataFrame for ML.
Returns X (features), y (target), enriched (full df).
Includes basic validation and a schema contract.
"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from typing import Tuple, List

EXPECTED_COLUMNS = [
    # common columns you may have in your loan datasets
    "client_name", "national_id", "loan_amount", "term_months", "status", "approval_date", "branch"
]

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def prepare_for_ml(df: pd.DataFrame, target_col: str = "defaulted") -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame]:
    df = _normalize_columns(df)
    # create target if missing
    if target_col not in df.columns:
        if "status" in df.columns:
            df[target_col] = df["status"].apply(lambda v: 1 if str(v).lower() in ("default","yes","1","true") else 0)
        else:
            df[target_col] = 0
    # date parsing
    for col in df.columns:
        if "date" in col or "dob" in col:
            try:
                df[col] = pd.to_datetime(df[col], errors="coerce")
                df[col + "_year"] = df[col].dt.year.fillna(0).astype(int)
                df[col + "_month"] = df[col].dt.month.fillna(0).astype(int)
            except Exception:
                pass
    # coerce numeric-like object columns
    for col in df.select_dtypes(include=["object"]).columns:
        s = df[col].astype(str)
        if s.str.match(r'^[\d\.\-\,\s]+$').any():
            try:
                df[col] = pd.to_numeric(s.str.replace(r'[^0-9\.\-]', '', regex=True), errors='coerce')
            except Exception:
                pass
    # fill common missing
    if "loan_amount" in df.columns:
        df["loan_amount"] = pd.to_numeric(df["loan_amount"], errors="coerce").fillna(0)
    # label encode low-cardinality categoricals
    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
    for c in list(cat_cols):
        df[c] = df[c].fillna("missing")
        if df[c].nunique() <= 200:
            try:
                le = LabelEncoder()
                df[c] = le.fit_transform(df[c].astype(str))
            except Exception:
                pass
        else:
            df.drop(columns=[c], inplace=True)
    # select numeric features
    numeric = df.select_dtypes(include=[np.number]).copy()
    if target_col in numeric.columns:
        numeric = numeric.drop(columns=[target_col])
    if numeric.shape[1] == 0:
        numeric["_dummy_feature"] = 1.0
    X = numeric.fillna(0)
    y = df[target_col].astype(int)
    return X, y, df

def validate_columns(df: pd.DataFrame) -> List[str]:
    dfcols = [c.strip().lower().replace(" ", "_") for c in df.columns]
    missing = [c for c in EXPECTED_COLUMNS if c not in dfcols]
    return missing
'''
safe_write(os.path.join(ROOT, "modules", "schema.py"), schema_py)

# -------------------------
# 5) modules/synth/generator.py (reuse your tuned Faker params)
# -------------------------
synth_py = r'''
"""
synth/generator.py - synthetic loan dataset generator.
This keeps the domain-specific Faker choices and branch list tuned by the user.
"""
from faker import Faker
import pandas as pd
import random
fake = Faker()

# Branches and distributions taken from user's configuration preferences
BRANCHES = ["Nairobi", "Mombasa", "Kisumu", "Eldoret", "Nakuru", "Garissa", "Nyeri"]

def generate_sample(n=1000, default_rate=0.08, seed: int = None):
    """
    Generates n synthetic loan records.
    default_rate: approximate proportion of defaulted loans
    seed: optional RNG seed for reproducibility
    """
    if seed is not None:
        random.seed(seed)
        Faker.seed(seed)
    records = []
    for i in range(n):
        name = fake.name()
        # national id style preserved per user notebook (9 digits)
        national_id = "".join(fake.random_choices(elements="0123456789", length=9))
        loan_amount = round(random.uniform(100, 100000), 2)
        term_months = random.choice([6, 12, 24, 36, 48, 60])
        # status weighted so 'current' majority and default according to default_rate
        status = random.choices(["current", "default", "late"], weights=[1-default_rate, default_rate*0.8, default_rate*0.2])[0]
        branch = random.choice(BRANCHES)
        approval_date = fake.date_between(start_date='-8y', end_date='today')
        records.append({
            "client_name": name,
            "national_id": national_id,
            "loan_amount": loan_amount,
            "term_months": term_months,
            "status": status,
            "branch": branch,
            "approval_date": approval_date
        })
    return pd.DataFrame.from_records(records)
'''
safe_write(os.path.join(ROOT, "modules", "synth", "generator.py"), synth_py)

# -------------------------
# 6) modules/ml/engine.py (tries XGBoost, falls back to LogisticRegression)
# -------------------------
ml_engine_py = r'''
"""
ml/engine.py - training pipeline. Tries to use XGBoost if available.
Saves model artifact + manifest JSON.
"""
import os, joblib, json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
MODEL_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "models"))
os.makedirs(MODEL_DIR, exist_ok=True)

def train_baseline(X, y, name="baseline", use_xgb=True):
    X = X.copy().fillna(0)
    try:
        strat = y if len(set(y)) > 1 else None
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=strat)
    except Exception:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model = None
    if use_xgb:
        try:
            import xgboost as xgb
            model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", n_jobs=1, random_state=42)
            model.fit(X_train, y_train)
        except Exception:
            model = None
    if model is None:
        from sklearn.linear_model import LogisticRegression
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
    # predictions
    if hasattr(model, "predict_proba"):
        preds = model.predict_proba(X_test)[:,1]
    else:
        preds = model.predict(X_test)
    auc = None
    try:
        if len(set(y_test)) > 1:
            auc = float(roc_auc_score(y_test, preds))
    except Exception:
        auc = None
    acc = float(accuracy_score(y_test, (preds > 0.5).astype(int))) if auc is not None else float(accuracy_score(y_test, model.predict(X_test)))
    model_path = os.path.join(MODEL_DIR, f"{name}.joblib")
    joblib.dump(model, model_path)
    meta = {"name": name, "auc": auc, "acc": acc, "n_features": int(X.shape[1])}
    with open(model_path + ".json", "w") as f:
        json.dump(meta, f)
    return model_path, meta
'''
safe_write(os.path.join(ROOT, "modules", "ml", "engine.py"), ml_engine_py)

# -------------------------
# 7) modules/ml/utils.py (SHAP wrapper, safe fallbacks)
# -------------------------
ml_utils_py = r'''
"""
ml/utils.py - helper for explainability (SHAP) but degrades gracefully if shap missing.
"""
import joblib, os
def explain_local(model_path, X, index=0, nsamples=100):
    try:
        import shap
        model = joblib.load(model_path)
        # prepare a small background
        background = X.sample(min(len(X), nsamples)) if hasattr(X, "sample") else X[:min(len(X), nsamples)]
        try:
            explainer = shap.Explainer(model, background)
            vals = explainer(X.iloc[[index]])
            return vals
        except Exception:
            # fallback to KernelExplainer (slow)
            try:
                explainer = shap.KernelExplainer(lambda x: model.predict_proba(x)[:,1], background)
                vals = explainer.shap_values(X.iloc[[index]])
                return vals
            except Exception:
                return None
    except Exception:
        return None
'''
safe_write(os.path.join(ROOT, "modules", "ml", "utils.py"), ml_utils_py)

# -------------------------
# 8) modules/audit.py & registry
# -------------------------
audit_py = r'''
"""
audit.py - append-only audit log and registry (JSON).
Uses atomic write patterns for safety.
"""
import os, json, time, tempfile
AUDIT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "data", "audit.log"))
REG_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "models", "registry.json"))

def _atomic_write(path, data: str):
    dname = os.path.dirname(path)
    os.makedirs(dname, exist_ok=True)
    fd, tmp = tempfile.mkstemp(dir=dname, prefix=".tmp")
    with os.fdopen(fd, "w") as f:
        f.write(data)
    os.replace(tmp, path)

def log_event(evt: dict):
    os.makedirs(os.path.dirname(AUDIT_PATH), exist_ok=True)
    evt2 = {"ts": time.time(), **evt}
    with open(AUDIT_PATH, "a") as f:
        f.write(json.dumps(evt2) + "\n")
    return evt2

def register_model(name, path, meta):
    data = {}
    if os.path.exists(REG_PATH):
        try:
            with open(REG_PATH) as f:
                data = json.load(f)
        except Exception:
            data = {}
    data[name] = {"path": path, "meta": meta, "registered_at": time.time()}
    _atomic_write(REG_PATH, json.dumps(data, indent=2))
    log_event({"action": "register_model", "name": name, "path": path})
    return True
'''
safe_write(os.path.join(ROOT, "modules", "audit.py"), audit_py)

# -------------------------
# 9) modules/reporting.py (simple per-client report exporter)
# -------------------------
reporting_py = r'''
"""
reporting.py - lightweight export helpers for client-level reports.
Exports CSV and a simple text summary. Can be replaced with richer PDF logic later.
"""
import io
import pandas as pd

def export_client_csv(df, client_id_col="national_id", client_id=None):
    if client_id is None:
        return None
    sub = df[df[client_id_col].astype(str) == str(client_id)]
    buf = io.StringIO()
    sub.to_csv(buf, index=False)
    return buf.getvalue()

def client_summary_text(df, client_id_col="national_id", client_id=None):
    sub = df[df[client_id_col].astype(str) == str(client_id)]
    if sub.empty:
        return "No records for client"
    tot_loans = len(sub)
    total_amount = sub.get("loan_amount", sub.get("amount", 0)).sum()
    defaults = sub[sub.get("status", "") == "default"].shape[0]
    return f"Client {client_id}: {tot_loans} loans, total amount {total_amount:.2f}, defaults {defaults}"
'''
safe_write(os.path.join(ROOT, "modules", "reporting.py"), reporting_py)

# -------------------------
# 10) modules/__init__.py and package inits
# -------------------------
safe_write(os.path.join(ROOT, "modules", "__init__.py"), "from . import auth, schema, synth, ml, audit, reporting\n")
safe_write(os.path.join(ROOT, "modules", "ml", "__init__.py"), "from . import engine, utils\n")
safe_write(os.path.join(ROOT, "modules", "synth", "__init__.py"), "from . import generator\n")

# -------------------------
# 11) Quick import check to ensure core modules load
# -------------------------
print("\n--- Import checks ---")
ok = True
failures = []
try:
    import importlib
    mods = ["modules.auth", "modules.schema", "modules.synth.generator", "modules.ml.engine", "modules.ml.utils", "modules.audit", "modules.reporting"]
    for m in mods:
        try:
            importlib.import_module(m)
            print("Imported", m)
        except Exception as e:
            print("FAILED import", m, "->", e)
            failures.append((m, str(e)))
            ok = False
except Exception as e:
    print("Unexpected import-check error:", e)
    ok = False

# final summary
print("\nBootstrap complete. ROOT:", ROOT)
if ok:
    print("All core modules imported successfully. Proceed to Cell 2 next.")
else:
    print("Some modules failed to import. Failures:", failures)
    print("If there are failures, copy the error messages here and I'll patch them before we proceed to Cell 2.")

Streamlit already installed; skipping package install (fast path).
Removing existing path for a clean bootstrap: /content/loan_app_full
Wrote: /content/loan_app_full/modules/auth.py
Wrote: /content/loan_app_full/modules/schema.py
Wrote: /content/loan_app_full/modules/synth/generator.py
Wrote: /content/loan_app_full/modules/ml/engine.py
Wrote: /content/loan_app_full/modules/ml/utils.py
Wrote: /content/loan_app_full/modules/audit.py
Wrote: /content/loan_app_full/modules/reporting.py
Wrote: /content/loan_app_full/modules/__init__.py
Wrote: /content/loan_app_full/modules/ml/__init__.py
Wrote: /content/loan_app_full/modules/synth/__init__.py

--- Import checks ---
Imported modules.auth
Imported modules.schema
Imported modules.synth.generator
Imported modules.ml.engine
Imported modules.ml.utils
Imported modules.audit
Imported modules.reporting

Bootstrap complete. ROOT: /content/loan_app_full
All core modules imported successfully. Proceed to Cell 2 next.


In [ ]:
import sys
sys.path.insert(0, "/content/loan_app_full")

# re-run import checks
import importlib
mods = [
    "modules.auth",
    "modules.schema",
    "modules.synth.generator",
    "modules.ml.engine",
    "modules.ml.utils",
    "modules.audit",
    "modules.reporting"
]
for m in mods:
    try:
        importlib.import_module(m)
        print("✅ Imported", m)
    except Exception as e:
        print("❌ FAILED", m, "->", e)

✅ Imported modules.auth
✅ Imported modules.schema
✅ Imported modules.synth.generator
✅ Imported modules.ml.engine
✅ Imported modules.ml.utils
✅ Imported modules.audit
✅ Imported modules.reporting


In [ ]:

# -------------------------
# Cell 2: Write Streamlit App with Path Fix + Admin Features
# -------------------------

from pathlib import Path
import textwrap

ROOT = Path("/content/loan_app_full").resolve()
app_dir = ROOT / "modules" / "streamlit_app"
app_dir.mkdir(parents=True, exist_ok=True)

app_code = textwrap.dedent(r'''
import sys
from pathlib import Path

# --- Path Fix for Streamlit ---
ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import streamlit as st
import pandas as pd
from modules import auth
from modules.synth import generator
from modules.schema import prepare_for_ml
from modules.ml import engine

# --- Initialize DB and Admin ---
auth.init_db()
conn = auth._conn(); cur = conn.cursor()
cur.execute("SELECT id FROM users WHERE username=?", ("Admin",))
if cur.fetchone() is None:
    cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                ("Admin", auth.sha256("Shady868"), "admin"))
else:
    cur.execute("UPDATE users SET password_hash=?, role=? WHERE username=?",
                (auth.sha256("Shady868"), "admin", "Admin"))
conn.commit(); conn.close()

# --- Session State ---
if "logged_in" not in st.session_state:
    st.session_state["logged_in"] = False
    st.session_state["username"] = None
    st.session_state["role"] = None

def login(u, r):
    st.session_state["logged_in"] = True
    st.session_state["username"] = u
    st.session_state["role"] = r

st.sidebar.title("LoanIQ Menu")
choice = st.sidebar.radio("Go to", ["Home", "Auth", "Upload", "Dashboard", "Admin Panel"])

# --- Auth Page ---
if choice == "Auth":
    st.header("Login / Register")
    col1, col2 = st.columns(2)

    # Login
    with col1:
        st.subheader("Login")
        u = st.text_input("Username")
        p = st.text_input("Password", type="password")
        if st.button("Login"):
            ok, role = auth.check_login(u, p)
            if ok:
                login(u, role)
                st.success("Logged in")
                st.experimental_rerun()

    # Register
    with col2:
        st.subheader("Register")
        ru = st.text_input("New user")
        rp = st.text_input("New pass", type="password")
        if st.button("Register"):
            ok, msg = auth.register_user(ru, rp)
            if ok:
                login(ru, "user")  # auto-login after registration
                st.success("Registered and logged in")
                st.experimental_rerun()

# --- Home Page ---
elif choice == "Home":
    st.header("Welcome to LoanIQ")
    if st.session_state["logged_in"]:
        st.success(f"Hello {st.session_state['username']} ({st.session_state['role']})")

# --- Upload Page ---
elif choice == "Upload":
    if not st.session_state["logged_in"]:
        st.warning("Please log in first.")
    else:
        up = st.file_uploader("Upload CSV", type=["csv"])
        if up:
            df = pd.read_csv(up)
            st.dataframe(df.head())

# --- Dashboard ---
elif choice == "Dashboard":
    if not st.session_state["logged_in"]:
        st.warning("Please log in first.")
    else:
        st.header("Generate Sample Data")
        if st.button("Generate"):
            df = generator.generate_sample(500)
            st.session_state["df"] = df
            st.success("Data generated")
        if "df" in st.session_state:
            st.dataframe(st.session_state["df"].head())

# --- Admin Panel ---
elif choice == "Admin Panel":
    if not st.session_state["logged_in"]:
        st.warning("Please log in first.")
    elif st.session_state["role"] != "admin":
        st.error("Access denied. Admins only.")
    else:
        st.header("Admin Controls")
        st.write("Impersonate a user, view logs, and manage accounts (placeholder).")
''')

# Write app.py
with open(app_dir / "app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print(f"✅ Streamlit app written to: {app_dir / 'app.py'}")

✅ Streamlit app written to: /content/loan_app_full/modules/streamlit_app/app.py


In [ ]:
# -------------------------
# Cell 3: Restart Streamlit + ngrok + Baseline Training (Fixed)
# -------------------------

import subprocess, time, json
from pathlib import Path

ROOT = Path("/content/loan_app_full").resolve()
app_path = ROOT / "modules" / "streamlit_app" / "app.py"

# -------------------------
# Kill old processes
# -------------------------
!pkill -f streamlit || true
!pkill -f ngrok || true

# -------------------------
# Start Streamlit with PYTHONPATH fix
# -------------------------
print("Starting Streamlit app with forced PYTHONPATH...")
streamlit_cmd = f"PYTHONPATH='{ROOT}' streamlit run '{app_path}' --server.port 8501 --server.headless true"
proc = subprocess.Popen(streamlit_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# -------------------------
# Start ngrok tunnel
# -------------------------
print("Starting ngrok tunnel...")
time.sleep(3)
subprocess.Popen(["ngrok", "http", "8501"])

# -------------------------
# Get ngrok public URL
# -------------------------
print("Waiting for ngrok URL...")
public_url = None
for _ in range(30):
    try:
        time.sleep(2)
        res = subprocess.check_output(["curl", "-s", "http://localhost:4040/api/tunnels"])
        data = json.loads(res.decode())
        tunnels = data.get("tunnels", [])
        if tunnels:
            public_url = tunnels[0].get("public_url")
            break
    except Exception:
        continue

if public_url:
    print(f"\n✅ Streamlit app is live at: {public_url}\n")
else:
    print("❌ Failed to get ngrok URL. Check logs.")

# -------------------------
# Baseline Model Training (with auto-create models dir)
# -------------------------
print("\nRunning baseline training job...\n")

try:
    from modules.synth import generator
    from modules.schema import prepare_for_ml
    from modules.ml import engine
    from modules import audit
    import pandas as pd

    # Ensure models directory exists
    models_dir = ROOT / "modules" / "models"
    models_dir.mkdir(parents=True, exist_ok=True)

    # Generate synthetic dataset (uses your Faker settings)
    df = generator.generate_sample(n=500)
    X, y, enriched = prepare_for_ml(df)

    # Train baseline model
    model_name = "baseline_model_auto"
    model_path, meta = engine.train_baseline(X, y, name=model_name, use_xgb=True)

    # Register model
    audit.register_model(model_name, model_path, meta)

    print("\n✅ Baseline model trained and registered.")
    print(f"Model path: {model_path}")
    print("Metadata:")
    print(json.dumps(meta, indent=2))
except Exception as e:
    print("❌ Baseline training failed:", e)

^C
^C
Starting Streamlit app with forced PYTHONPATH...
Starting ngrok tunnel...
Waiting for ngrok URL...

✅ Streamlit app is live at: https://759200b039b0.ngrok-free.app


Running baseline training job...


✅ Baseline model trained and registered.
Model path: /content/loan_app_full/modules/models/baseline_model_auto.joblib
Metadata:
{
  "name": "baseline_model_auto",
  "auc": 0.9824455205811138,
  "acc": 0.96,
  "n_features": 7
}


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [19:43:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
import json
from pathlib import Path

ROOT = Path("/content/loan_app_full").resolve()
config_path = ROOT / "config.json"

data = {
    "ngrok_token": "31rYvgklL0EdX9bGLvTXc313efE_2GyDFGPUNAyFgB83bikTF"
}

with open(config_path, "w", encoding="utf-8") as f:
    json.dump(data, f)

print(f"✅ config.json created at {config_path}")

✅ config.json created at /content/loan_app_full/config.json


In [ ]:
!grep -rl "st.experimental_rerun" /content/loan_app_full/modules/streamlit_app | xargs sed -i 's/st.experimental_rerun()/st.rerun()/g'

In [ ]:
auth_file = "/content/loan_app_full/modules/auth.py"

with open(auth_file, "r", encoding="utf-8") as f:
    code = f.read()

# Insert auto-login after "Registration successful" logic
if "Registration successful" in code and "session_state" not in code:
    patch = """
st.session_state["logged_in"] = True
st.session_state["username"] = username
st.success("Registration successful! You are now logged in.")
st.rerun()
"""
    code = code.replace('st.success("Registration successful")', patch)

with open(auth_file, "w", encoding="utf-8") as f:
    f.write(code)

print("✅ Auto-login patch applied to auth.py")

✅ Auto-login patch applied to auth.py
